The data contains a pair of paragraphs. These text paragraphs are randomly sampled from a raw dataset. Each pair of the sentence may or may not be semantically similar. The dataset considered for this project does not contain any labels. Given below is the solution to the unsupervised machine learning problem

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import nltk
import re
import scipy
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from gensim.models import FastText

In [ ]:
data =  pd.read_csv("../input/retail-questions/Questions.csv", sep='\t')
print(data.head())
sentences = data["Questions"]
print(sentences)


# Preprocessing of Data
*Removing Special Char, stopwords&lemmitizing*

In [ ]:
lemmatizer=WordNetLemmatizer()
stop_words=stopwords.words("english")

def remove(sent_r):
    
    #conver all letters to lower case
    sent_r=sent_r.lower()
    
    #remove special character 
    no_spec=re.sub("[^A-Za-z0-9 " "]","",sent_r)
    
    temp=' '
    #lemmitizing and removing stopwords
    temp=temp.join([lemmatizer.lemmatize(i) for i in no_spec.split(" ")])
    l1=temp.split(" ")
 
    #removing extra spaces
    while("" in l1):
        l1.remove("")
    return l1

#a=remove("Ra^*vic&h&&[andar] () ram &((*&^))scores [Goo##d] runs")
#print(a)
#output ['ravichandar', 'ram', 'score', 'good', 'run']

# FastText
FastText is an extension to Word2Vec proposed by Facebook in 2016. Instead of feeding individual words into the Neural Network, FastText breaks words into several n-grams (sub-words). For instance, the tri-grams for the word apple is app, ppl, and ple (ignoring the starting and ending of boundaries of words). The word embedding vector for apple will be the sum of all these n-grams. After training the Neural Network, we will have word embeddings for all the n-grams given the training dataset. Rare words can now be properly represented since it is highly likely that some of their n-grams also appears in other words.


In [ ]:
model = FastText('https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz')


# Vector calculation

In [ ]:
def vect(s):
    return np.mean([model[word] for word in s],axis=0)
        

# Calculating vector for each sentence in dataframe and saving it in a new column

In [ ]:
vect_frame=[]
for i in sentences:
    #cleaning and preprocessing
    list_clean=remove(i)
    
    #coverts sent to vec
    vector1=vect(list_clean)
    vect_frame.append(vector1)
    
#store vectors in data frame
data["vectors"]=vect_frame

## Cosine Similarity
Cosine similarity is a measure of similarity between two non-zero vectors of an inner product space that measures the cosine of the angle between them. The cosine of 0° is 1, and it is less than 1 for any angle in the interval (0, π] radians.


# Calculating cosine similarity for all Vectors in a dataframe and saving it in new column

In [ ]:
def CompareBoth(vector_single_sentence):
    result=[]
    for i in range(len(data["vectors"])):
        cosine = scipy.spatial.distance.cosine(vector_single_sentence, data["vectors"][i])
        result.append((1-cosine)*100)
    
    #save all results to the dataframe
    data["results"]=result

# Getting a sentence from user and displaying Top 5 similar sentences

In [ ]:
a=input("Enter a Text: ") #gets a sentence from user
print("\n")
list_clean=remove(a) #cleaning and preprocessing
print(list_clean)
vector_single_sentence=vect(list_clean) # covert to vec
top=CompareBoth(vector_single_sentence)

#Printing Top 5 similar sentences
aa=data['results'].nlargest(n=5)
offset=0

for i in aa.keys():
    if data["results"][i]>=offset:
        print(data["Questions"][i],' ',data["results"][i])